In [2]:
%pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

Note: you may need to restart the kernel to use updated packages.
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/mssmartypants/rice-type-classification


100%|██████████| 888k/888k [00:00<00:00, 570MB/s]

In [5]:
%pip install torchsummary


Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch 
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [7]:
data_df = pd.read_csv("/Users/tkaur/Desktop/ML_Learning/PyTorch_Learning/rice-type-classification/riceClassification.csv")
data_df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [8]:
data_df.dropna(inplace=True)
data_df.drop('id', axis=1, inplace=True)
print(data_df.shape)

(18185, 11)


In [9]:
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [10]:
print(data_df["Class"].unique())

[1 0]


In [11]:
print(data_df["Class"].value_counts())

Class
1    9985
0    8200
Name: count, dtype: int64


In [12]:
original_df = data_df.copy()
for column in data_df.columns:
    data_df[column] = data_df[column] / data_df[column].abs().max()
    
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [13]:
X = np.array(data_df.iloc[:, :-1])
Y = np.array(data_df.iloc[:,-1])

In [ ]:
# splitting the dataset into training
# here since testing dataset iss 30%, then training is 70%

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
# splitting the dataset for validation
# dividing the 30% testing dataset into validation and testing equally 
# so 30% testing goes 50% to validation which is 15% and 50% goes to testing which is 15%
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [ ]:
print(X_train.shape)  # 70% data for training
print(X_val.shape)      # then dividing the rest of the 30% into validation and testing equally, so here it is 15% validation
print(X_test.shape)     # and 15% testing data

(12729, 10)
(2728, 10)
(2728, 10)


In [18]:
class dataset(Dataset):         # creating a dataset object
    def __init__(self, X, Y):       # creating a constructor here
        self.X = torch.tensor(X, dtype=torch.float32).to(device)
        self.Y = torch.tensor(Y, dtype=torch.float32).to(device)
        
    def __len__(self):
        return len(self.X)      # returns the shape of the input
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]

In [19]:
training_data = dataset(X_train, y_train)
validation_data = dataset(X_val, y_val)
testing_data = dataset(X_test, y_test)

In [20]:
train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=8, shuffle=True)
testing_dataloader = DataLoader(testing_data, batch_size=8, shuffle=True) 

In [21]:
for x, y in train_dataloader:
    print(x)
    print("======")
    print(y)
    break

tensor([[0.8382, 0.8563, 0.8513, 0.9248, 0.8040, 0.9155, 0.7884, 0.7504, 0.8164,
         0.5706],
        [0.9025, 0.8997, 0.8730, 0.9303, 0.8600, 0.9500, 0.6277, 0.7688, 0.8373,
         0.5847],
        [0.8091, 0.8289, 0.8496, 0.9175, 0.7630, 0.8995, 0.8874, 0.7093, 0.8819,
         0.5536],
        [0.6931, 0.7766, 0.7752, 0.9239, 0.6553, 0.8326, 0.6795, 0.6643, 0.8614,
         0.5684],
        [0.4671, 0.7414, 0.5481, 0.9753, 0.4435, 0.6834, 0.9123, 0.5968, 0.7192,
         0.7674],
        [0.5506, 0.7999, 0.5995, 0.9736, 0.5278, 0.7420, 0.5694, 0.6484, 0.7183,
         0.7569],
        [0.6535, 0.8685, 0.6615, 0.9715, 0.6302, 0.8084, 0.6235, 0.7073, 0.7164,
         0.7448],
        [0.5733, 0.7830, 0.6482, 0.9597, 0.5469, 0.7571, 0.6241, 0.6379, 0.7726,
         0.6854]])
tensor([0., 0., 0., 0., 1., 1., 1., 1.])
